# Split documents into chunks for RAG

Break PDFs and documents into searchable chunks for retrieval-augmented generation (RAG) pipelines.

## Problem

You have PDF documents or text files that you want to use for retrieval-augmented generation (RAG). Before you can search them, you need to:

1. Split documents into smaller chunks
2. Generate embeddings for each chunk
3. Store everything in a searchable index

| Document | Size | Chunks needed |
|----------|------|---------------|
| annual_report.pdf | 50 pages | ~100 chunks |
| user_manual.pdf | 20 pages | ~40 chunks |
| research_paper.pdf | 10 pages | ~20 chunks |

## Solution

**What's in this recipe:**
- Split PDFs into paragraphs or sentences
- Control chunk size with token limits
- Add embeddings for semantic search

You create a view with a DocumentSplitter iterator that automatically breaks documents into chunks. Then you add an embedding index for semantic search.

### Setup

In [ ]:
%pip install -qU pixeltable sentence-transformers

In [ ]:
import pixeltable as pxt
from pixeltable.iterators import DocumentSplitter
from pixeltable.functions.huggingface import sentence_transformer

### Load documents

In [ ]:
# Create a fresh directory
pxt.drop_dir('rag_demo', force=True)
pxt.create_dir('rag_demo')

In [ ]:
# Create table for documents
docs = pxt.create_table('rag_demo.documents', {'document': pxt.Document})

In [ ]:
# Insert a sample PDF
docs.insert([
    {'document': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/rag-demo/Argus-Market-Digest-June-2024.pdf'}
])

### Split into chunks

Create a view that splits each document into paragraphs with a token limit:

In [ ]:
# Create a view that splits documents into chunks
chunks = pxt.create_view(
    'rag_demo.chunks',
    docs,
    iterator=DocumentSplitter.create(
        document=docs.document,
        separators='paragraph,token_limit',  # Split by paragraph with token limit
        limit=300  # Max 300 tokens per chunk
    )
)

In [ ]:
# View the chunks
chunks.select(chunks.text).head(5)

### Add semantic search

Create an embedding index on the chunks for similarity search:

In [ ]:
# Add embedding index for semantic search
chunks.add_embedding_index(
    column='text',
    embed=sentence_transformer.using(model_id='all-MiniLM-L6-v2')
)

### Search your documents

Use similarity search to find relevant chunks:

In [ ]:
# Search for relevant chunks
query = "market trends"
sim = chunks.text.similarity(query)

results = (
    chunks
    .order_by(sim, asc=False)
    .select(chunks.text, score=sim)
    .limit(3)
)
results.collect()

## Explanation

**Separator options:**

| Separator | Description |
|-----------|-------------|
| `paragraph` | Split on paragraph breaks |
| `sentence` | Split on sentence boundaries |
| `heading` | Split on document headings |
| `page` | Split on page breaks |
| `token_limit` | Split at token count only |

You can combine separators: `separators='paragraph,token_limit'`

**Chunk sizing:**

- `limit`: Maximum tokens per chunk (default: 500)
- `overlap`: Tokens to overlap between chunks (default: 0)

**New documents are processed automatically:**

When you insert new documents, chunks and embeddings are generated without extra code.

## See also

- [Iterators documentation](https://docs.pixeltable.com/platform/iterators)
- [RAG demo notebook](https://docs.pixeltable.com/howto/use-cases/rag-demo)